# **Undirected Unweighted Graphs - Complete Guide**

An **Undirected Unweighted Graph** is a collection of vertices (nodes) connected by edges where edges have no direction and no associated weight. This means if there's an edge between vertex A and vertex B, you can travel in both directions: A ↔ B.

**Key Features:**
- **Vertices (Nodes)**: Individual points or entities in the graph
- **Undirected Edges**: Connections without specific direction (A ↔ B)
- **Symmetric Relations**: Edge (u,v) implies edge (v,u) exists
- **Degree**: Number of edges connected to a vertex
- **Path**: Sequence of vertices connected by edges
- **Cycle**: Path that starts and ends at the same vertex
- **Connected**: Path exists between every pair of vertices

**Graph Terminology:**
- **Adjacent Vertices**: Two vertices connected by an edge
- **Degree of Vertex**: Number of edges incident to the vertex
- **Connected Graph**: Path exists between any two vertices
- **Connected Component**: Maximal connected subgraph
- **Tree**: Connected acyclic graph with V-1 edges
- **Forest**: Collection of trees (acyclic graph)
- **Bipartite**: Vertices can be colored with two colors
- **Complete Graph**: Every pair of vertices is connected

**Applications:**
- **Social Networks**: Friendships, mutual connections
- **Computer Networks**: Ethernet, mesh networks  
- **Transportation**: Roads, railway systems
- **Circuit Design**: Electrical connections
- **Molecular Structure**: Chemical bonds
- **Game Theory**: Player interactions
- **Image Processing**: Pixel neighborhoods
- **Collaboration Networks**: Research partnerships

## Graph Structure and Node Definition

A **Node (Vertex)** in an undirected graph represents an entity that can have symmetric connections with other nodes. Each node maintains:

- **Node Identity**: Unique identifier (name, number, etc.)
- **Adjacent Nodes**: All nodes directly connected to this node
- **Degree**: Count of edges connected to this node
- **Traversal State**: Visited status, discovery times, colors (for algorithms)

In undirected graphs, edges are symmetric: if A connects to B, then B connects to A. This symmetry simplifies many algorithms but requires careful handling to avoid double-counting edges.

In [1]:
import random
from collections import defaultdict, deque
import sys

# Create a sample undirected unweighted graph with 8 nodes
nodes = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H']

# Define undirected edges for our sample graph
# Format: (node1, node2) - order doesn't matter, connection is bidirectional
edges = [
    ('A', 'B'), ('A', 'C'), ('A', 'D'),
    ('B', 'E'), ('B', 'F'),
    ('C', 'F'), ('C', 'G'),
    ('D', 'G'),
    ('E', 'F'),
    ('G', 'H'),
    # Disconnected component
    # ('I', 'J') - will add later for component analysis
]

print("=== Sample Undirected Unweighted Graph ===")
print(f"Nodes: {nodes}")
print(f"Undirected edges:")
for u, v in edges:
    print(f"  {u} ↔ {v}")

print(f"\nTotal Nodes: {len(nodes)}")
print(f"Total Undirected Edges: {len(edges)}")

# Calculate degree for each node
degree = {node: 0 for node in nodes}

for u, v in edges:
    degree[u] += 1
    degree[v] += 1

print("Node degrees:")
for node in sorted(nodes):
    print(f"  {node}: degree = {degree[node]}")

# Basic graph properties
total_degrees = sum(degree.values())
print(f"\nGraph Properties:")
print(f"Sum of all degrees: {total_degrees}")
print(f"Handshaking lemma verification: {total_degrees} = 2 × {len(edges)} = {2 * len(edges)}")
print(f"Average degree: {total_degrees / len(nodes):.2f}")

# Find isolated vertices
isolated = [node for node in nodes if degree[node] == 0]
if isolated:
    print(f"Isolated vertices: {isolated}")
else:
    print("No isolated vertices")

=== Sample Undirected Unweighted Graph ===
Nodes: ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H']
Undirected edges:
  A ↔ B
  A ↔ C
  A ↔ D
  B ↔ E
  B ↔ F
  C ↔ F
  C ↔ G
  D ↔ G
  E ↔ F
  G ↔ H

Total Nodes: 8
Total Undirected Edges: 10
Node degrees:
  A: degree = 3
  B: degree = 3
  C: degree = 3
  D: degree = 2
  E: degree = 2
  F: degree = 3
  G: degree = 3
  H: degree = 1

Graph Properties:
Sum of all degrees: 20
Handshaking lemma verification: 20 = 2 × 10 = 20
Average degree: 2.50
No isolated vertices


## Graph Representations (Matrix vs List)

Undirected graphs have symmetric relationships, which affects both representation methods:

### 1. Adjacency Matrix
- **Symmetric matrix** where `matrix[i][j] = matrix[j][i] = 1` if there's an edge
- **Main diagonal** typically zeros (no self-loops in simple graphs)
- **Space optimization**: Only store upper/lower triangle for simple graphs

### 2. Adjacency List  
- **Dictionary/List** where each vertex stores a list of all adjacent vertices
- **Each edge appears twice**: once in each vertex's adjacency list
- **More space-efficient** for sparse graphs

**Time Complexity Comparison:**

| Operation | Adjacency Matrix | Adjacency List |
|-----------|------------------|----------------|
| Add Edge | O(1) | O(1) |
| Remove Edge | O(1) | O(degree) |
| Check Edge | O(1) | O(degree) |
| Get Neighbors | O(V) | O(degree) |
| Space Complexity | O(V²) | O(V + E) |

**Memory Usage:**
- **Matrix**: Always V² space regardless of edge count
- **List**: V + 2E space (each edge stored twice)
- **Crossover point**: Adjacency list better when E << V²/2

In [2]:
class UndirectedGraph:
    def __init__(self, nodes):
        self.nodes = nodes
        self.node_to_index = {node: i for i, node in enumerate(nodes)}
        self.num_nodes = len(nodes)
        
        # Adjacency Matrix representation (symmetric)
        self.adj_matrix = [[0 for _ in range(self.num_nodes)] for _ in range(self.num_nodes)]
        
        # Adjacency List representation
        self.adj_list = defaultdict(list)
        
        # Keep track of edges for analysis
        self.edges = set()
    
    def add_edge(self, u, v):
        """Add undirected edge between vertices u and v"""
        u_idx = self.node_to_index[u]
        v_idx = self.node_to_index[v]
        
        # Update adjacency matrix (symmetric)
        self.adj_matrix[u_idx][v_idx] = 1
        self.adj_matrix[v_idx][u_idx] = 1
        
        # Update adjacency lists (both directions)
        if v not in self.adj_list[u]:
            self.adj_list[u].append(v)
        if u not in self.adj_list[v]:
            self.adj_list[v].append(u)
        
        # Store edge (normalize order for consistency)
        edge = tuple(sorted([u, v]))
        self.edges.add(edge)
    
    def remove_edge(self, u, v):
        """Remove undirected edge between vertices u and v"""
        u_idx = self.node_to_index[u]
        v_idx = self.node_to_index[v]
        
        # Update adjacency matrix
        self.adj_matrix[u_idx][v_idx] = 0
        self.adj_matrix[v_idx][u_idx] = 0
        
        # Update adjacency lists
        if v in self.adj_list[u]:
            self.adj_list[u].remove(v)
        if u in self.adj_list[v]:
            self.adj_list[v].remove(u)
        
        # Remove edge
        edge = tuple(sorted([u, v]))
        self.edges.discard(edge)
    
    def has_edge(self, u, v):
        """Check if edge exists between u and v"""
        u_idx = self.node_to_index[u]
        v_idx = self.node_to_index[v]
        return self.adj_matrix[u_idx][v_idx] == 1
    
    def get_neighbors(self, node):
        """Get all neighbors of a node"""
        return list(self.adj_list[node])
    
    def get_degree(self, node):
        """Get degree of a node"""
        return len(self.adj_list[node])
    
    def display_matrix(self):
        """Display adjacency matrix representation"""
        print("Adjacency Matrix (symmetric):")
        print("     ", end="")
        for node in self.nodes:
            print(f"{node:>3}", end="")
        print()
        
        for i, node in enumerate(self.nodes):
            print(f"{node:>2}: ", end="")
            for j in range(self.num_nodes):
                print(f"{self.adj_matrix[i][j]:>2}", end=" ")
            print()
    
    def display_list(self):
        """Display adjacency list representation"""
        print("Adjacency List:")
        for node in sorted(self.nodes):
            neighbors = sorted(self.adj_list[node])
            print(f"{node} ↔ {neighbors}")
    
    def get_graph_stats(self):
        """Get basic statistics about the graph"""
        total_edges = len(self.edges)
        degrees = [self.get_degree(node) for node in self.nodes]
        
        stats = {
            'vertices': self.num_nodes,
            'edges': total_edges,
            'max_edges': self.num_nodes * (self.num_nodes - 1) // 2,
            'density': 2 * total_edges / (self.num_nodes * (self.num_nodes - 1)) if self.num_nodes > 1 else 0,
            'avg_degree': sum(degrees) / self.num_nodes if self.num_nodes > 0 else 0,
            'max_degree': max(degrees) if degrees else 0,
            'min_degree': min(degrees) if degrees else 0
        }
        return stats

# Create and populate our sample undirected graph
graph = UndirectedGraph(nodes)

# Add all undirected edges
for u, v in edges:
    graph.add_edge(u, v)

print("=== Graph Representations ===")
graph.display_matrix()
print()
graph.display_list()
print()

# Display graph statistics
stats = graph.get_graph_stats()
print("=== Graph Statistics ===")
print(f"Vertices: {stats['vertices']}")
print(f"Edges: {stats['edges']}")
print(f"Maximum possible edges: {stats['max_edges']}")
print(f"Graph density: {stats['density']:.3f}")
print(f"Average degree: {stats['avg_degree']:.2f}")
print(f"Degree range: {stats['min_degree']} - {stats['max_degree']}")

print(f"\nDetailed degree analysis:")
for node in sorted(nodes):
    degree = graph.get_degree(node)
    neighbors = graph.get_neighbors(node)
    print(f"{node}: degree={degree}, neighbors={sorted(neighbors)}")

# Test matrix vs list space efficiency
matrix_space = stats['vertices'] * stats['vertices']
list_space = stats['vertices'] + 2 * stats['edges']  # Each edge stored twice

print(f"\n=== Space Efficiency Analysis ===")
print(f"Adjacency Matrix: {matrix_space} entries")
print(f"Adjacency List: {list_space} entries")
print(f"Better choice: {'Matrix' if matrix_space <= list_space else 'List'}")
print(f"Space ratio (List/Matrix): {list_space/matrix_space:.3f}")

=== Graph Representations ===
Adjacency Matrix (symmetric):
       A  B  C  D  E  F  G  H
 A:  0  1  1  1  0  0  0  0 
 B:  1  0  0  0  1  1  0  0 
 C:  1  0  0  0  0  1  1  0 
 D:  1  0  0  0  0  0  1  0 
 E:  0  1  0  0  0  1  0  0 
 F:  0  1  1  0  1  0  0  0 
 G:  0  0  1  1  0  0  0  1 
 H:  0  0  0  0  0  0  1  0 

Adjacency List:
A ↔ ['B', 'C', 'D']
B ↔ ['A', 'E', 'F']
C ↔ ['A', 'F', 'G']
D ↔ ['A', 'G']
E ↔ ['B', 'F']
F ↔ ['B', 'C', 'E']
G ↔ ['C', 'D', 'H']
H ↔ ['G']

=== Graph Statistics ===
Vertices: 8
Edges: 10
Maximum possible edges: 28
Graph density: 0.357
Average degree: 2.50
Degree range: 1 - 3

Detailed degree analysis:
A: degree=3, neighbors=['B', 'C', 'D']
B: degree=3, neighbors=['A', 'E', 'F']
C: degree=3, neighbors=['A', 'F', 'G']
D: degree=2, neighbors=['A', 'G']
E: degree=2, neighbors=['B', 'F']
F: degree=3, neighbors=['B', 'C', 'E']
G: degree=3, neighbors=['C', 'D', 'H']
H: degree=1, neighbors=['G']

=== Space Efficiency Analysis ===
Adjacency Matrix: 64 entries
A

## Breadth-First Search (BFS) Traversal

**BFS** in undirected graphs explores vertices level by level, naturally finding the **shortest path** between any two vertices in terms of number of edges. Since edges are bidirectional, BFS can reach all vertices in the same connected component.

**Algorithm Steps:**
1. Start from source vertex, mark it visited
2. Add source to queue with distance 0
3. While queue is not empty:
   - Dequeue vertex
   - Visit all unvisited neighbors
   - Mark neighbors as visited and enqueue with distance + 1
4. Process vertices in the order they are dequeued

**Key Properties:**
- **Level-order traversal**: Visits all distance-k vertices before distance-(k+1)
- **Shortest path**: Guarantees minimum hop count between vertices
- **Connected component**: Discovers exactly one connected component
- **Tree structure**: BFS creates a spanning tree of the component

**Time Complexity:** O(V + E) where V is vertices and E is edges
**Space Complexity:** O(V) for the queue and visited set

**Use Cases:**
- **Shortest hop distance** in unweighted graphs
- **Level-order exploration** of networks
- **Connected component** identification
- **Social network analysis** (degrees of separation)

In [3]:
def bfs_undirected(graph, start_node):
    """
    Breadth-First Search traversal of undirected graph
    Returns BFS order, distances, parent pointers, and spanning tree edges
    """
    visited = set()
    queue = deque([(start_node, 0)])  # (node, distance)
    bfs_order = []
    distances = {start_node: 0}
    parent = {start_node: None}
    tree_edges = []
    
    visited.add(start_node)
    
    print(f"Starting BFS from node: {start_node}")
    print("Step by step traversal:")
    
    step = 1
    while queue:
        current_node, current_dist = queue.popleft()
        bfs_order.append(current_node)
        
        print(f"Step {step}: Visit {current_node} (distance: {current_dist})")
        
        # Get unvisited neighbors
        new_neighbors = []
        for neighbor in sorted(graph.get_neighbors(current_node)):
            if neighbor not in visited:
                new_neighbors.append(neighbor)
                visited.add(neighbor)
                queue.append((neighbor, current_dist + 1))
                distances[neighbor] = current_dist + 1
                parent[neighbor] = current_node
                tree_edges.append((current_node, neighbor))
        
        if new_neighbors:
            print(f"         Add to queue: {new_neighbors}")
        
        queue_nodes = [node for node, _ in queue]
        print(f"         Queue state: {queue_nodes}")
        step += 1
        print()
    
    return bfs_order, distances, parent, tree_edges

def bfs_shortest_path(graph, start, end):
    """Find shortest path between two nodes using BFS"""
    if start == end:
        return [start], 0
    
    _, distances, parent, _ = bfs_undirected(graph, start)
    
    if end not in distances:
        return None, float('inf')  # No path exists
    
    # Reconstruct path
    path = []
    current = end
    while current is not None:
        path.append(current)
        current = parent[current]
    
    path.reverse()
    return path, distances[end]

def bfs_all_distances(graph, start):
    """Find distances from start to all reachable nodes"""
    _, distances, _, _ = bfs_undirected(graph, start)
    
    reachable = set(distances.keys())
    unreachable = set(graph.nodes) - reachable
    
    return distances, reachable, unreachable

# Perform BFS traversal from different starting points
print("=== BFS Traversal Analysis ===")

for start in ['A', 'E', 'H']:
    print(f"\n{'='*50}")
    bfs_result, bfs_distances, bfs_parent, bfs_tree = bfs_undirected(graph, start)
    
    print(f"BFS Results from {start}:")
    print(f"  Traversal order: {' → '.join(bfs_result)}")
    print(f"  Distances: {bfs_distances}")
    print(f"  BFS tree edges: {bfs_tree}")
    
    # Check reachability
    reachable = set(bfs_distances.keys())
    unreachable = set(graph.nodes) - reachable
    
    if unreachable:
        print(f"  Unreachable nodes: {sorted(unreachable)}")
        print(f"  → Graph has multiple connected components")
    else:
        print(f"  → All nodes reachable (single connected component)")

# Test shortest path functionality
print(f"\n=== Shortest Path Examples ===")
path_tests = [('A', 'H'), ('B', 'G'), ('A', 'F'), ('E', 'D')]

for start, end in path_tests:
    path, distance = bfs_shortest_path(graph, start, end)
    if path:
        print(f"{start} → {end}: {' → '.join(path)} (distance: {distance})")
    else:
        print(f"{start} → {end}: No path exists")

# Analyze BFS tree properties
print(f"\n=== BFS Tree Analysis ===")
bfs_order, distances, parent, tree_edges = bfs_undirected(graph, 'A')

print(f"BFS spanning tree from A:")
print(f"  Tree edges: {tree_edges}")
print(f"  Number of tree edges: {len(tree_edges)}")
print(f"  Expected for connected component: V-1 = {len(bfs_order)-1}")

# Verify tree property
if len(tree_edges) == len(bfs_order) - 1:
    print(f"  ✓ Valid spanning tree")
else:
    print(f"  ✗ Not a spanning tree (disconnected component)")

print(f"  Tree levels:")
levels = defaultdict(list)
for node, dist in distances.items():
    levels[dist].append(node)

for level in sorted(levels.keys()):
    print(f"    Level {level}: {sorted(levels[level])}")

=== BFS Traversal Analysis ===

Starting BFS from node: A
Step by step traversal:
Step 1: Visit A (distance: 0)
         Add to queue: ['B', 'C', 'D']
         Queue state: ['B', 'C', 'D']

Step 2: Visit B (distance: 1)
         Add to queue: ['E', 'F']
         Queue state: ['C', 'D', 'E', 'F']

Step 3: Visit C (distance: 1)
         Add to queue: ['G']
         Queue state: ['D', 'E', 'F', 'G']

Step 4: Visit D (distance: 1)
         Queue state: ['E', 'F', 'G']

Step 5: Visit E (distance: 2)
         Queue state: ['F', 'G']

Step 6: Visit F (distance: 2)
         Queue state: ['G']

Step 7: Visit G (distance: 2)
         Add to queue: ['H']
         Queue state: ['H']

Step 8: Visit H (distance: 3)
         Queue state: []

BFS Results from A:
  Traversal order: A → B → C → D → E → F → G → H
  Distances: {'A': 0, 'B': 1, 'C': 1, 'D': 1, 'E': 2, 'F': 2, 'G': 2, 'H': 3}
  BFS tree edges: [('A', 'B'), ('A', 'C'), ('A', 'D'), ('B', 'E'), ('B', 'F'), ('C', 'G'), ('G', 'H')]
  → All nodes

## Depth-First Search (DFS) Traversal

**DFS** in undirected graphs explores as far as possible along each path before backtracking. It naturally creates a **spanning tree** and can detect **cycles** in the graph.

### DFS Variants for Undirected Graphs:

### 1. Recursive DFS
- **Natural implementation** using function call stack
- **Easy to understand** and implement
- **Space complexity** depends on recursion depth

### 2. Iterative DFS  
- **Explicit stack** to avoid recursion limits
- **More control** over traversal order
- **Constant space** overhead

### 3. DFS with Edge Classification
- **Tree edges**: Edges in the DFS spanning tree
- **Back edges**: Connect vertex to ancestor (indicate cycles)
- **No forward/cross edges** in undirected graphs

**Time Complexity:** O(V + E) 
**Space Complexity:** O(V) for visited tracking + O(V) for recursion stack

**Use Cases:**
- **Path finding** with backtracking
- **Cycle detection** in undirected graphs
- **Connected component** identification
- **Topological problems** on trees
- **Maze solving** and puzzle games

In [4]:
def dfs_recursive(graph, start_node, visited=None, order=None, parent=None, tree_edges=None, back_edges=None):
    """
    Recursive DFS traversal with edge classification
    """
    if visited is None:
        visited = set()
        order = []
        parent = {node: None for node in graph.nodes}
        tree_edges = []
        back_edges = []
        print(f"Starting DFS from node: {start_node}")
        print("Step by step traversal:")
    
    visited.add(start_node)
    order.append(start_node)
    print(f"Visit {start_node}")
    
    # Visit all unvisited neighbors
    for neighbor in sorted(graph.get_neighbors(start_node)):
        if neighbor not in visited:
            # Tree edge
            tree_edges.append((start_node, neighbor))
            parent[neighbor] = start_node
            print(f"  Tree edge: {start_node} → {neighbor}")
            dfs_recursive(graph, neighbor, visited, order, parent, tree_edges, back_edges)
        
        elif parent[start_node] != neighbor:
            # Back edge (avoid parent edge which would always be "back")
            if (neighbor, start_node) not in back_edges:  # Avoid duplicate back edges
                back_edges.append((start_node, neighbor))
                print(f"  Back edge: {start_node} → {neighbor} (cycle detected)")
    
    return order, parent, tree_edges, back_edges

def dfs_iterative(graph, start_node):
    """
    Iterative DFS traversal using explicit stack
    """
    visited = set()
    stack = [start_node]
    order = []
    parent = {node: None for node in graph.nodes}
    tree_edges = []
    
    print(f"Starting iterative DFS from node: {start_node}")
    print("Step by step traversal:")
    
    while stack:
        current = stack.pop()
        
        if current not in visited:
            visited.add(current)
            order.append(current)
            print(f"Visit {current}")
            
            # Add neighbors to stack (reverse order to match recursive DFS)
            neighbors = sorted(graph.get_neighbors(current), reverse=True)
            for neighbor in neighbors:
                if neighbor not in visited:
                    stack.append(neighbor)
                    if parent[neighbor] is None:  # Set parent for first discovery
                        parent[neighbor] = current
                        tree_edges.append((current, neighbor))
            
            if neighbors:
                unvisited_neighbors = [n for n in neighbors if n not in visited]
                if unvisited_neighbors:
                    print(f"  Add to stack: {unvisited_neighbors}")
        
        print(f"  Stack state: {stack}")
    
    return order, parent, tree_edges

def dfs_path_finding(graph, start, end, path=None):
    """
    Find a path between two nodes using DFS
    """
    if path is None:
        path = []
    
    path = path + [start]
    
    if start == end:
        return path
    
    for neighbor in graph.get_neighbors(start):
        if neighbor not in path:  # Avoid cycles
            new_path = dfs_path_finding(graph, neighbor, end, path)
            if new_path:
                return new_path
    
    return None

def dfs_all_paths(graph, start, end, path=None, all_paths=None):
    """
    Find all simple paths between two nodes using DFS
    """
    if path is None:
        path = []
        all_paths = []
    
    path = path + [start]
    
    if start == end:
        all_paths.append(path)
        return all_paths
    
    for neighbor in graph.get_neighbors(start):
        if neighbor not in path:  # Avoid cycles
            dfs_all_paths(graph, neighbor, end, path, all_paths)
    
    return all_paths

# Perform recursive DFS traversal
print("=== Recursive DFS Traversal ===")
dfs_order, dfs_parent, dfs_tree_edges, dfs_back_edges = dfs_recursive(graph, 'A')

print(f"\nRecursive DFS Results:")
print(f"  Traversal order: {' → '.join(dfs_order)}")
print(f"  Tree edges: {dfs_tree_edges}")
print(f"  Back edges: {dfs_back_edges}")
print(f"  Cycles detected: {'Yes' if dfs_back_edges else 'No'}")

# Perform iterative DFS traversal
print(f"\n{'='*50}")
print("=== Iterative DFS Traversal ===")
iter_order, iter_parent, iter_tree_edges = dfs_iterative(graph, 'A')

print(f"\nIterative DFS Results:")
print(f"  Traversal order: {' → '.join(iter_order)}")
print(f"  Tree edges: {iter_tree_edges}")

# Compare recursive vs iterative
print(f"\n=== Comparison: Recursive vs Iterative ===")
print(f"Same traversal order: {dfs_order == iter_order}")
print(f"Same tree edges: {set(dfs_tree_edges) == set(iter_tree_edges)}")

# Test path finding capabilities
print(f"\n=== DFS Path Finding ===")
path_tests = [('A', 'H'), ('B', 'D'), ('E', 'G')]

for start, end in path_tests:
    # Find one path
    single_path = dfs_path_finding(graph, start, end)
    print(f"\nPath from {start} to {end}:")
    if single_path:
        print(f"  Single path: {' → '.join(single_path)}")
        
        # Find all paths
        all_paths = dfs_all_paths(graph, start, end)
        print(f"  All simple paths ({len(all_paths)}):")
        for i, path in enumerate(all_paths, 1):
            print(f"    Path {i}: {' → '.join(path)}")
    else:
        print(f"  No path exists")

# Analyze DFS tree structure
print(f"\n=== DFS Tree Analysis ===")
print(f"DFS spanning tree from A:")
print(f"  Number of tree edges: {len(dfs_tree_edges)}")
print(f"  Expected for connected component: {len(dfs_order) - 1}")

# Build tree structure
children = defaultdict(list)
for parent_node, child in dfs_tree_edges:
    children[parent_node].append(child)

def print_tree(node, level=0, printed=None):
    if printed is None:
        printed = set()
    
    if node in printed:
        return
    
    printed.add(node)
    print("  " * level + f"├─ {node}")
    
    for child in sorted(children[node]):
        print_tree(child, level + 1, printed)

print(f"  DFS tree structure:")
print_tree('A')

=== Recursive DFS Traversal ===
Starting DFS from node: A
Step by step traversal:
Visit A
  Tree edge: A → B
Visit B
  Tree edge: B → E
Visit E
  Tree edge: E → F
Visit F
  Back edge: F → B (cycle detected)
  Tree edge: F → C
Visit C
  Back edge: C → A (cycle detected)
  Tree edge: C → G
Visit G
  Tree edge: G → D
Visit D
  Back edge: D → A (cycle detected)
  Tree edge: G → H
Visit H

Recursive DFS Results:
  Traversal order: A → B → E → F → C → G → D → H
  Tree edges: [('A', 'B'), ('B', 'E'), ('E', 'F'), ('F', 'C'), ('C', 'G'), ('G', 'D'), ('G', 'H')]
  Back edges: [('F', 'B'), ('C', 'A'), ('D', 'A')]
  Cycles detected: Yes

=== Iterative DFS Traversal ===
Starting iterative DFS from node: A
Step by step traversal:
Visit A
  Add to stack: ['D', 'C', 'B']
  Stack state: ['D', 'C', 'B']
Visit B
  Add to stack: ['F', 'E']
  Stack state: ['D', 'C', 'F', 'E']
Visit E
  Add to stack: ['F']
  Stack state: ['D', 'C', 'F', 'F']
Visit F
  Add to stack: ['C']
  Stack state: ['D', 'C', 'F', 'C']


## Connected Components Analysis

**Connected Components** are maximal sets of vertices where every vertex is reachable from every other vertex in the set via undirected paths. Finding connected components is fundamental for understanding graph structure.

### Algorithms for Finding Connected Components:

### 1. DFS-based Component Finding
- **Run DFS** from each unvisited vertex
- **Each DFS traversal** discovers one complete component
- **Mark all reached vertices** as visited

### 2. BFS-based Component Finding
- **Similar approach** using BFS instead of DFS
- **Level-order discovery** of each component
- **Same time complexity** as DFS approach

### 3. Union-Find (Disjoint Set Union)
- **Efficient for dynamic** component tracking
- **Near-constant time** operations with path compression
- **Useful when edges are added incrementally**

**Properties of Connected Components:**
- **Partition**: Every vertex belongs to exactly one component
- **Maximality**: Cannot add more vertices to a component
- **Bridge edges**: Edges connecting different components don't exist

**Time Complexity:** O(V + E) for DFS/BFS approaches
**Space Complexity:** O(V) for visited tracking and component storage

**Use Cases:**
- **Network analysis**: Finding isolated subnetworks
- **Social networks**: Identifying communities or groups
- **Image processing**: Connected pixel regions
- **Circuit analysis**: Identifying separate circuits

In [5]:
def find_connected_components_dfs(graph):
    """
    Find all connected components using DFS
    """
    visited = set()
    components = []
    component_id = {}
    
    def dfs_component(node, current_component, comp_id):
        visited.add(node)
        current_component.append(node)
        component_id[node] = comp_id
        
        for neighbor in graph.get_neighbors(node):
            if neighbor not in visited:
                dfs_component(neighbor, current_component, comp_id)
    
    print("=== Finding Connected Components using DFS ===")
    comp_id = 0
    
    for node in sorted(graph.nodes):
        if node not in visited:
            current_component = []
            print(f"Component {comp_id + 1}: Starting DFS from {node}")
            dfs_component(node, current_component, comp_id)
            components.append(current_component)
            print(f"  Found component: {sorted(current_component)}")
            comp_id += 1
    
    return components, component_id

def find_connected_components_bfs(graph):
    """
    Find all connected components using BFS
    """
    visited = set()
    components = []
    component_id = {}
    
    def bfs_component(start_node, comp_id):
        queue = deque([start_node])
        current_component = []
        visited.add(start_node)
        
        while queue:
            node = queue.popleft()
            current_component.append(node)
            component_id[node] = comp_id
            
            for neighbor in graph.get_neighbors(node):
                if neighbor not in visited:
                    visited.add(neighbor)
                    queue.append(neighbor)
        
        return current_component
    
    print("=== Finding Connected Components using BFS ===")
    comp_id = 0
    
    for node in sorted(graph.nodes):
        if node not in visited:
            print(f"Component {comp_id + 1}: Starting BFS from {node}")
            current_component = bfs_component(node, comp_id)
            components.append(current_component)
            print(f"  Found component: {sorted(current_component)}")
            comp_id += 1
    
    return components, component_id

def analyze_components(graph, components, component_id):
    """
    Analyze the structure and properties of connected components
    """
    print(f"\n=== Component Analysis ===")
    print(f"Number of connected components: {len(components)}")
    
    # Component sizes
    component_sizes = [len(comp) for comp in components]
    print(f"Component sizes: {component_sizes}")
    
    # Analyze each component
    for i, component in enumerate(components):
        comp_nodes = set(component)
        
        # Count edges within component
        internal_edges = 0
        for node in component:
            for neighbor in graph.get_neighbors(node):
                if neighbor in comp_nodes and node < neighbor:  # Avoid double counting
                    internal_edges += 1
        
        # Calculate component properties
        n = len(component)
        max_edges = n * (n - 1) // 2
        density = internal_edges / max_edges if max_edges > 0 else 0
        
        print(f"\nComponent {i + 1}: {sorted(component)}")
        print(f"  Vertices: {n}")
        print(f"  Edges: {internal_edges}")
        print(f"  Maximum possible edges: {max_edges}")
        print(f"  Density: {density:.3f}")
        
        # Check if component is a tree
        if internal_edges == n - 1:
            print(f"  Type: Tree (connected and acyclic)")
        elif internal_edges < n - 1:
            print(f"  Type: Forest (disconnected)")
        else:
            print(f"  Type: Graph with cycles")
    
    return component_sizes

def create_disconnected_graph():
    """
    Create a graph with multiple connected components for testing
    """
    # Add isolated nodes to create disconnected components
    extended_nodes = nodes + ['I', 'J', 'K']
    disconnected_edges = edges + [('I', 'J')]  # K remains isolated
    
    disconnected_graph = UndirectedGraph(extended_nodes)
    for u, v in disconnected_edges:
        disconnected_graph.add_edge(u, v)
    
    return disconnected_graph

def graph_connectivity_properties(graph, components):
    """
    Analyze overall connectivity properties of the graph
    """
    print(f"\n=== Graph Connectivity Properties ===")
    
    num_components = len(components)
    total_nodes = len(graph.nodes)
    total_edges = len(graph.edges)
    
    # Basic connectivity
    is_connected = num_components == 1
    print(f"Connected: {is_connected}")
    
    if is_connected:
        print(f"Graph is connected - single component with {total_nodes} vertices")
        
        # For connected graphs, analyze tree vs cyclic nature
        if total_edges == total_nodes - 1:
            print(f"Graph is a tree (V-1 = {total_nodes-1} edges)")
        elif total_edges < total_nodes - 1:
            print(f"Graph is a forest (fewer than V-1 edges)")
        else:
            print(f"Graph has cycles ({total_edges - (total_nodes - 1)} extra edges)")
    
    else:
        print(f"Graph is disconnected - {num_components} components")
        
        # Calculate minimum edges needed to connect
        edges_to_connect = num_components - 1
        print(f"Minimum edges needed to connect: {edges_to_connect}")
    
    # Component distribution
    component_sizes = [len(comp) for comp in components]
    largest_component_size = max(component_sizes)
    largest_component_fraction = largest_component_size / total_nodes
    
    print(f"Largest component: {largest_component_size} vertices ({largest_component_fraction:.1%})")
    
    # Isolated vertices
    isolated_vertices = [comp[0] for comp in components if len(comp) == 1]
    print(f"Isolated vertices: {len(isolated_vertices)} {isolated_vertices if isolated_vertices else ''}")

# Test connected components on original graph
print("=== Testing on Original Graph ===")
components_dfs, comp_id_dfs = find_connected_components_dfs(graph)
component_sizes = analyze_components(graph, components_dfs, comp_id_dfs)
graph_connectivity_properties(graph, components_dfs)

# Test on disconnected graph
print(f"\n{'='*60}")
print("=== Testing on Disconnected Graph ===")
disconnected_graph = create_disconnected_graph()

print("Extended graph with disconnected components:")
disconnected_graph.display_list()

components_dfs_disc, comp_id_dfs_disc = find_connected_components_dfs(disconnected_graph)
print()
components_bfs_disc, comp_id_bfs_disc = find_connected_components_bfs(disconnected_graph)

# Verify both methods find same components
print(f"\n=== Method Comparison ===")
print(f"DFS found {len(components_dfs_disc)} components")
print(f"BFS found {len(components_bfs_disc)} components")

# Normalize component lists for comparison
dfs_comp_sets = [set(comp) for comp in components_dfs_disc]
bfs_comp_sets = [set(comp) for comp in components_bfs_disc]
dfs_comp_sets.sort(key=len, reverse=True)
bfs_comp_sets.sort(key=len, reverse=True)

print(f"Same components found: {dfs_comp_sets == bfs_comp_sets}")

analyze_components(disconnected_graph, components_dfs_disc, comp_id_dfs_disc)
graph_connectivity_properties(disconnected_graph, components_dfs_disc)

=== Testing on Original Graph ===
=== Finding Connected Components using DFS ===
Component 1: Starting DFS from A
  Found component: ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H']

=== Component Analysis ===
Number of connected components: 1
Component sizes: [8]

Component 1: ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H']
  Vertices: 8
  Edges: 10
  Maximum possible edges: 28
  Density: 0.357
  Type: Graph with cycles

=== Graph Connectivity Properties ===
Connected: True
Graph is connected - single component with 8 vertices
Graph has cycles (3 extra edges)
Largest component: 8 vertices (100.0%)
Isolated vertices: 0 

=== Testing on Disconnected Graph ===
Extended graph with disconnected components:
Adjacency List:
A ↔ ['B', 'C', 'D']
B ↔ ['A', 'E', 'F']
C ↔ ['A', 'F', 'G']
D ↔ ['A', 'G']
E ↔ ['B', 'F']
F ↔ ['B', 'C', 'E']
G ↔ ['C', 'D', 'H']
H ↔ ['G']
I ↔ ['J']
J ↔ ['I']
K ↔ []
=== Finding Connected Components using DFS ===
Component 1: Starting DFS from A
  Found component: ['A', 'B', 'C', 'D', 

## Bipartite Graph Detection

A **Bipartite Graph** is a graph whose vertices can be divided into two disjoint sets such that no two vertices within the same set are adjacent. This is equivalent to being **2-colorable**.

### Properties of Bipartite Graphs:
- **Two-colorable**: Vertices can be colored with exactly 2 colors
- **No odd cycles**: Contains no cycles of odd length
- **Perfect matching**: May have perfect matchings between the two sets
- **Planar**: All bipartite graphs are planar

### Algorithm: 2-Coloring with BFS/DFS
1. **Start from any unvisited vertex**, color it with color 0
2. **Color all neighbors** with the opposite color (1)
3. **Continue BFS/DFS**, alternating colors
4. **If conflict found** (neighbor has same color), graph is not bipartite
5. **Repeat for all components** in disconnected graphs

**Time Complexity:** O(V + E)
**Space Complexity:** O(V) for color storage and queue/stack

**Applications:**
- **Matching problems**: Job assignments, dating apps
- **Scheduling**: Tasks and resources, time slots
- **Network analysis**: Actors and movies, customers and products
- **Conflict resolution**: Ensuring no conflicts within groups

In [6]:
def is_bipartite_bfs(graph):
    """
    Check if graph is bipartite using BFS coloring
    Returns: (is_bipartite, coloring, partition)
    """
    color = {}  # -1: uncolored, 0: color A, 1: color B
    
    def bfs_color_component(start_node):
        queue = deque([start_node])
        color[start_node] = 0  # Start with color 0
        
        print(f"  Starting BFS coloring from {start_node}")
        
        while queue:
            current = queue.popleft()
            current_color = color[current]
            opposite_color = 1 - current_color
            
            print(f"    Processing {current} (color {current_color})")
            
            for neighbor in graph.get_neighbors(current):
                if neighbor not in color:
                    # Color with opposite color
                    color[neighbor] = opposite_color
                    queue.append(neighbor)
                    print(f"      Color {neighbor} with {opposite_color}")
                
                elif color[neighbor] == current_color:
                    # Same color conflict - not bipartite
                    print(f"      CONFLICT: {current} and {neighbor} both have color {current_color}")
                    return False
        
        return True
    
    print("=== Bipartite Detection using BFS ===")
    
    # Process all connected components
    for node in sorted(graph.nodes):
        if node not in color:
            print(f"Component starting from {node}:")
            if not bfs_color_component(node):
                return False, color, None
    
    # Create partition
    set_a = [node for node, c in color.items() if c == 0]
    set_b = [node for node, c in color.items() if c == 1]
    
    return True, color, (sorted(set_a), sorted(set_b))

def is_bipartite_dfs(graph):
    """
    Check if graph is bipartite using DFS coloring
    """
    color = {}
    
    def dfs_color(node, node_color):
        color[node] = node_color
        print(f"    Color {node} with {node_color}")
        
        for neighbor in graph.get_neighbors(node):
            if neighbor not in color:
                # Recursively color with opposite color
                if not dfs_color(neighbor, 1 - node_color):
                    return False
            elif color[neighbor] == node_color:
                # Same color conflict
                print(f"      CONFLICT: {node} and {neighbor} both have color {node_color}")
                return False
        
        return True
    
    print("=== Bipartite Detection using DFS ===")
    
    for node in sorted(graph.nodes):
        if node not in color:
            print(f"Component starting from {node}:")
            if not dfs_color(node, 0):
                return False, color, None
    
    # Create partition
    set_a = [node for node, c in color.items() if c == 0]
    set_b = [node for node, c in color.items() if c == 1]
    
    return True, color, (sorted(set_a), sorted(set_b))

def verify_bipartite_properties(graph, is_bipartite, coloring, partition):
    """
    Verify bipartite properties and analyze the partition
    """
    print(f"\n=== Bipartite Verification ===")
    print(f"Graph is bipartite: {is_bipartite}")
    
    if not is_bipartite:
        print("Graph contains odd cycles or coloring conflicts")
        return
    
    set_a, set_b = partition
    print(f"Partition A: {set_a}")
    print(f"Partition B: {set_b}")
    print(f"Partition sizes: |A| = {len(set_a)}, |B| = {len(set_b)}")
    
    # Verify no edges within partitions
    edges_within_a = 0
    edges_within_b = 0
    edges_between = 0
    
    for u, v in graph.edges:
        u_color = coloring[u]
        v_color = coloring[v]
        
        if u_color == v_color:
            if u_color == 0:
                edges_within_a += 1
            else:
                edges_within_b += 1
            print(f"ERROR: Edge {u}-{v} within same partition!")
        else:
            edges_between += 1
    
    print(f"Edges within A: {edges_within_a}")
    print(f"Edges within B: {edges_within_b}")
    print(f"Edges between A and B: {edges_between}")
    print(f"Total edges: {len(graph.edges)}")
    
    # Calculate maximum possible edges in bipartite graph
    max_bipartite_edges = len(set_a) * len(set_b)
    density = edges_between / max_bipartite_edges if max_bipartite_edges > 0 else 0
    
    print(f"Maximum possible edges in bipartite graph: {max_bipartite_edges}")
    print(f"Bipartite density: {density:.3f}")

def create_bipartite_test_graphs():
    """
    Create test graphs to demonstrate bipartite detection
    """
    # Test 1: Simple bipartite graph
    bipartite_nodes = ['A', 'B', 'C', 'D', 'E', 'F']
    bipartite_edges = [('A', 'D'), ('A', 'E'), ('B', 'D'), ('B', 'F'), ('C', 'E'), ('C', 'F')]
    
    bipartite_graph = UndirectedGraph(bipartite_nodes)
    for u, v in bipartite_edges:
        bipartite_graph.add_edge(u, v)
    
    # Test 2: Non-bipartite graph (contains odd cycle)
    nonbipartite_nodes = ['X', 'Y', 'Z']
    nonbipartite_edges = [('X', 'Y'), ('Y', 'Z'), ('Z', 'X')]  # Triangle (odd cycle)
    
    nonbipartite_graph = UndirectedGraph(nonbipartite_nodes)
    for u, v in nonbipartite_edges:
        nonbipartite_graph.add_edge(u, v)
    
    return bipartite_graph, nonbipartite_graph

def detect_odd_cycles(graph):
    """
    Detect odd cycles in the graph (which prevent bipartiteness)
    """
    print(f"\n=== Odd Cycle Detection ===")
    
    color = {}
    parent = {}
    odd_cycles = []
    
    def dfs_odd_cycle(node, node_color, node_parent):
        color[node] = node_color
        parent[node] = node_parent
        
        for neighbor in graph.get_neighbors(node):
            if neighbor == node_parent:
                continue  # Skip parent edge
            
            if neighbor not in color:
                if dfs_odd_cycle(neighbor, 1 - node_color, node):
                    return True
            elif color[neighbor] == node_color:
                # Found odd cycle
                print(f"Odd cycle detected involving edge {node}-{neighbor}")
                return True
        
        return False
    
    for node in graph.nodes:
        if node not in color:
            if dfs_odd_cycle(node, 0, None):
                return True
    
    return False

# Test bipartite detection on original graph
print("=== Testing Bipartite Detection on Original Graph ===")
is_bip_bfs, coloring_bfs, partition_bfs = is_bipartite_bfs(graph)
print()
is_bip_dfs, coloring_dfs, partition_dfs = is_bipartite_dfs(graph)

print(f"\n=== Method Comparison ===")
print(f"BFS result: {is_bip_bfs}")
print(f"DFS result: {is_bip_dfs}")
print(f"Same result: {is_bip_bfs == is_bip_dfs}")

if is_bip_bfs:
    verify_bipartite_properties(graph, is_bip_bfs, coloring_bfs, partition_bfs)
else:
    detect_odd_cycles(graph)

# Test on specifically designed graphs
print(f"\n{'='*60}")
print("=== Testing on Designed Test Graphs ===")

bipartite_test, nonbipartite_test = create_bipartite_test_graphs()

# Test bipartite graph
print("\n--- Testing Known Bipartite Graph ---")
print("Graph structure:")
bipartite_test.display_list()

is_bip, coloring, partition = is_bipartite_bfs(bipartite_test)
verify_bipartite_properties(bipartite_test, is_bip, coloring, partition)

# Test non-bipartite graph
print(f"\n--- Testing Known Non-Bipartite Graph ---")
print("Graph structure:")
nonbipartite_test.display_list()

is_bip, coloring, partition = is_bipartite_bfs(nonbipartite_test)
verify_bipartite_properties(nonbipartite_test, is_bip, coloring, partition)
detect_odd_cycles(nonbipartite_test)

# Additional analysis
print(f"\n=== Bipartite Graph Applications ===")
print("Common applications of bipartite graphs:")
print("1. Matching problems: Jobs ↔ Candidates")
print("2. Network analysis: Users ↔ Products")  
print("3. Scheduling: Tasks ↔ Time slots")
print("4. Database design: Primary tables ↔ Junction tables")
print("5. Social networks: People ↔ Events")

=== Testing Bipartite Detection on Original Graph ===
=== Bipartite Detection using BFS ===
Component starting from A:
  Starting BFS coloring from A
    Processing A (color 0)
      Color B with 1
      Color C with 1
      Color D with 1
    Processing B (color 1)
      Color E with 0
      Color F with 0
    Processing C (color 1)
      Color G with 0
    Processing D (color 1)
    Processing E (color 0)
      CONFLICT: E and F both have color 0

=== Bipartite Detection using DFS ===
Component starting from A:
    Color A with 0
    Color B with 1
    Color E with 0
    Color F with 1
      CONFLICT: F and B both have color 1

=== Method Comparison ===
BFS result: False
DFS result: False
Same result: True

=== Odd Cycle Detection ===
Odd cycle detected involving edge F-B

=== Testing on Designed Test Graphs ===

--- Testing Known Bipartite Graph ---
Graph structure:
Adjacency List:
A ↔ ['D', 'E']
B ↔ ['D', 'F']
C ↔ ['E', 'F']
D ↔ ['A', 'B']
E ↔ ['A', 'C']
F ↔ ['B', 'C']
=== Bipartit

## Cycle Detection in Undirected Graphs

**Cycle detection** in undirected graphs is simpler than in directed graphs since we only need to find any closed path. A cycle exists if there's a path from a vertex back to itself without repeating edges.

### Detection Methods:

### 1. DFS with Parent Tracking
- **Track parent** of each vertex during DFS
- **If we visit a neighbor** that's not the parent, we found a cycle
- **Simple and efficient** for undirected graphs

### 2. Union-Find (Disjoint Set Union)
- **Union vertices** connected by edges
- **If vertices are already connected**, adding edge creates cycle
- **Efficient for incremental** edge addition

### 3. BFS with Parent Tracking
- **Similar to DFS approach** but using BFS
- **Level-order cycle detection**

**Key Insight for Undirected Graphs:**
- **Back edge to non-parent** indicates a cycle
- **No need for complex coloring** like in directed graphs
- **Tree edges vs Back edges** classification suffices

**Time Complexity:** O(V + E) for DFS/BFS, O(E α(V)) for Union-Find
**Space Complexity:** O(V) for visited tracking

**Applications:**
- **Network loop detection** in computer networks
- **Chemical structure analysis** (ring detection)
- **Circuit design** (avoiding short circuits)
- **Social network analysis** (finding closed groups)

In [7]:
def detect_cycle_dfs(graph):
    """
    Detect cycles using DFS with parent tracking
    Returns: (has_cycle, cycle_edges, cycle_path)
    """
    visited = set()
    parent = {}
    cycle_edges = []
    cycle_paths = []
    
    def dfs_cycle(node, node_parent):
        visited.add(node)
        parent[node] = node_parent
        
        print(f"  Visit {node} (parent: {node_parent})")
        
        for neighbor in sorted(graph.get_neighbors(node)):
            if neighbor == node_parent:
                continue  # Skip parent edge to avoid false positive
            
            if neighbor not in visited:
                # Tree edge - continue DFS
                print(f"    Tree edge: {node} - {neighbor}")
                if dfs_cycle(neighbor, node):
                    return True
            else:
                # Back edge found - cycle detected
                print(f"    Back edge: {node} - {neighbor} (CYCLE FOUND)")
                cycle_edges.append((node, neighbor))
                
                # Reconstruct cycle path
                cycle_path = [neighbor]
                current = node
                while current != neighbor:
                    cycle_path.append(current)
                    current = parent[current]
                cycle_path.append(neighbor)  # Close the cycle
                
                cycle_paths.append(cycle_path)
                return True
        
        return False
    
    print("=== Cycle Detection using DFS ===")
    
    for node in sorted(graph.nodes):
        if node not in visited:
            print(f"Starting DFS from {node}")
            if dfs_cycle(node, None):
                break
    
    has_cycle = len(cycle_edges) > 0
    return has_cycle, cycle_edges, cycle_paths

class UnionFind:
    """
    Union-Find (Disjoint Set Union) data structure
    with path compression and union by rank
    """
    def __init__(self, nodes):
        self.parent = {node: node for node in nodes}
        self.rank = {node: 0 for node in nodes}
    
    def find(self, node):
        """Find root with path compression"""
        if self.parent[node] != node:
            self.parent[node] = self.find(self.parent[node])  # Path compression
        return self.parent[node]
    
    def union(self, u, v):
        """Union by rank"""
        root_u = self.find(u)
        root_v = self.find(v)
        
        if root_u == root_v:
            return False  # Already in same set (cycle found)
        
        # Union by rank
        if self.rank[root_u] < self.rank[root_v]:
            self.parent[root_u] = root_v
        elif self.rank[root_u] > self.rank[root_v]:
            self.parent[root_v] = root_u
        else:
            self.parent[root_v] = root_u
            self.rank[root_u] += 1
        
        return True
    
    def get_components(self):
        """Get all connected components"""
        components = defaultdict(list)
        for node in self.parent:
            root = self.find(node)
            components[root].append(node)
        return dict(components)

def detect_cycle_union_find(graph):
    """
    Detect cycles using Union-Find data structure
    """
    uf = UnionFind(graph.nodes)
    cycle_edges = []
    
    print("=== Cycle Detection using Union-Find ===")
    
    for u, v in sorted(graph.edges):
        print(f"Processing edge {u} - {v}")
        
        if not uf.union(u, v):
            print(f"  CYCLE FOUND: Edge {u} - {v} creates a cycle")
            cycle_edges.append((u, v))
            break
        else:
            print(f"  Added to forest")
    
    has_cycle = len(cycle_edges) > 0
    components = uf.get_components()
    
    return has_cycle, cycle_edges, components

def find_all_cycles_dfs(graph):
    """
    Find all fundamental cycles in the graph
    Uses spanning tree approach
    """
    visited = set()
    tree_edges = set()
    back_edges = []
    all_cycles = []
    parent = {}
    
    def dfs_all_cycles(node, node_parent):
        visited.add(node)
        parent[node] = node_parent
        
        for neighbor in graph.get_neighbors(node):
            if neighbor == node_parent:
                continue
            
            edge = tuple(sorted([node, neighbor]))
            
            if neighbor not in visited:
                # Tree edge
                tree_edges.add(edge)
                dfs_all_cycles(neighbor, node)
            else:
                # Back edge (if not already processed)
                if edge not in tree_edges and (neighbor, node) not in back_edges:
                    back_edges.append((node, neighbor))
    
    # Build spanning forest
    for node in graph.nodes:
        if node not in visited:
            dfs_all_cycles(node, None)
    
    print(f"=== Finding All Fundamental Cycles ===")
    print(f"Tree edges: {sorted(tree_edges)}")
    print(f"Back edges: {back_edges}")
    
    # Each back edge creates exactly one fundamental cycle
    for back_u, back_v in back_edges:
        # Find path in tree from back_u to back_v
        cycle_path = []
        
        # Find path from back_u to root
        path_u = []
        current = back_u
        while current is not None:
            path_u.append(current)
            current = parent[current]
        
        # Find path from back_v to root
        path_v = []
        current = back_v
        while current is not None:
            path_v.append(current)
            current = parent[current]
        
        # Find lowest common ancestor
        ancestors_u = set(path_u)
        lca = None
        for node in path_v:
            if node in ancestors_u:
                lca = node
                break
        
        if lca is not None:
            # Build cycle: back_u -> lca -> back_v -> back_u
            cycle = []
            
            # Path from back_u to lca
            current = back_u
            while current != lca:
                cycle.append(current)
                current = parent[current]
            cycle.append(lca)
            
            # Path from lca to back_v (reverse)
            path_to_v = []
            current = back_v
            while current != lca:
                path_to_v.append(current)
                current = parent[current]
            
            cycle.extend(reversed(path_to_v))
            all_cycles.append((cycle, (back_u, back_v)))
    
    return all_cycles, tree_edges, back_edges

def create_cyclic_test_graph():
    """
    Create a graph with known cycles for testing
    """
    cyclic_nodes = ['A', 'B', 'C', 'D', 'E']
    cyclic_edges = [
        ('A', 'B'), ('B', 'C'), ('C', 'A'),  # Triangle cycle
        ('C', 'D'), ('D', 'E'), ('E', 'C')   # Another cycle
    ]
    
    cyclic_graph = UndirectedGraph(cyclic_nodes)
    for u, v in cyclic_edges:
        cyclic_graph.add_edge(u, v)
    
    return cyclic_graph

# Test cycle detection on original graph
print("=== Testing Cycle Detection on Original Graph ===")
has_cycle_dfs, cycle_edges_dfs, cycle_paths_dfs = detect_cycle_dfs(graph)

print(f"\nDFS Cycle Detection Results:")
print(f"Has cycle: {has_cycle_dfs}")
if has_cycle_dfs:
    print(f"Cycle edges: {cycle_edges_dfs}")
    print(f"Cycle paths: {cycle_paths_dfs}")

print()
has_cycle_uf, cycle_edges_uf, components_uf = detect_cycle_union_find(graph)

print(f"\nUnion-Find Cycle Detection Results:")
print(f"Has cycle: {has_cycle_uf}")
if has_cycle_uf:
    print(f"Cycle edges: {cycle_edges_uf}")

print(f"Final components: {components_uf}")

# Test on graph with known cycles
print(f"\n{'='*60}")
print("=== Testing on Graph with Multiple Cycles ===")

cyclic_test_graph = create_cyclic_test_graph()
print("Test graph structure:")
cyclic_test_graph.display_list()

has_cycle, cycle_edges, cycle_paths = detect_cycle_dfs(cyclic_test_graph)
print(f"\nCycle detection results:")
print(f"Has cycle: {has_cycle}")
if has_cycle:
    print(f"First cycle found: {cycle_paths[0] if cycle_paths else 'None'}")

# Find all cycles
all_cycles, tree_edges, back_edges = find_all_cycles_dfs(cyclic_test_graph)
print(f"\nAll fundamental cycles:")
for i, (cycle, back_edge) in enumerate(all_cycles, 1):
    print(f"Cycle {i}: {' → '.join(cycle)} → {cycle[0]} (back edge: {back_edge})")

print(f"\nGraph structure analysis:")
print(f"Total edges: {len(cyclic_test_graph.edges)}")
print(f"Tree edges: {len(tree_edges)}")
print(f"Back edges: {len(back_edges)}")
print(f"Fundamental cycles: {len(all_cycles)}")
print(f"Expected cycles (E - V + C): {len(cyclic_test_graph.edges) - len(cyclic_test_graph.nodes) + len(components_uf)}")

=== Testing Cycle Detection on Original Graph ===
=== Cycle Detection using DFS ===
Starting DFS from A
  Visit A (parent: None)
    Tree edge: A - B
  Visit B (parent: A)
    Tree edge: B - E
  Visit E (parent: B)
    Tree edge: E - F
  Visit F (parent: E)
    Back edge: F - B (CYCLE FOUND)

DFS Cycle Detection Results:
Has cycle: True
Cycle edges: [('F', 'B')]
Cycle paths: [['B', 'F', 'E', 'B']]

=== Cycle Detection using Union-Find ===
Processing edge A - B
  Added to forest
Processing edge A - C
  Added to forest
Processing edge A - D
  Added to forest
Processing edge B - E
  Added to forest
Processing edge B - F
  Added to forest
Processing edge C - F
  CYCLE FOUND: Edge C - F creates a cycle

Union-Find Cycle Detection Results:
Has cycle: True
Cycle edges: [('C', 'F')]
Final components: {'A': ['A', 'B', 'C', 'D', 'E', 'F'], 'G': ['G'], 'H': ['H']}

=== Testing on Graph with Multiple Cycles ===
Test graph structure:
Adjacency List:
A ↔ ['B', 'C']
B ↔ ['A', 'C']
C ↔ ['A', 'B', 'D',

## Shortest Path Algorithms

In **unweighted undirected graphs**, **BFS** is the optimal algorithm for finding shortest paths since all edges have the same weight (1). The shortest path between any two vertices is the path with the minimum number of edges.

### Single-Source Shortest Paths:
- **BFS from source** finds shortest distances to all reachable vertices
- **Path reconstruction** using parent pointers
- **Guarantees optimality** for unweighted graphs

### All-Pairs Shortest Paths:
- **Run BFS from each vertex** for complete distance matrix
- **Floyd-Warshall** can be used but BFS is more efficient for unweighted graphs
- **Space-time tradeoff** between computing on-demand vs pre-computing

### Path Properties in Undirected Graphs:
- **Symmetric distances**: dist(u,v) = dist(v,u)
- **Connected components**: Infinite distance between disconnected vertices
- **Diameter**: Longest shortest path in the graph
- **Radius**: Minimum eccentricity of any vertex

**Time Complexity:** 
- Single-source: O(V + E)
- All-pairs: O(V × (V + E)) = O(V² + VE)

**Space Complexity:** O(V) for single-source, O(V²) for all-pairs

**Applications:**
- **Social networks**: Degrees of separation
- **Navigation systems**: Minimum hop routing
- **Network analysis**: Communication efficiency
- **Game AI**: Optimal movement in grid-based games